In [ ]:
import os
import geopandas as gp
import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator
import matplotlib.gridspec as gridspec
from mpl_toolkits.basemap import Basemap
from pylab import *
import matplotlib.ticker as ticker
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [ ]:
plt.rcParams['axes.unicode_minus']=False#负号
plt.rcParams['font.sans-serif']=['Arial']###############
plt.rcParams['font.size'] = '7' # 设置字体大小 = '16' # 设置字体大小
# plt.rcParams["font.weight"] = "bold"
# plt.rcParams["axes.labelweight"] = "bold"
plt.rcParams['xtick.direction'] = 'in'#将x周的刻度线方向设置向内
plt.rcParams['ytick.direction'] = 'in'#将y轴的刻度方向设置向内

In [ ]:
# latlon_1degree_SJ_last=path+'/global_grid.shp'
version='v16_250104'
output_dir = os.path.join(r'J:\lakemapping\postprocess',version)
excel_dir=os.path.join(output_dir,f'stastics_excel')

gdb=os.path.join(output_dir,f'7_correct_temp_file.gdb')
fig_path=r'D:\lakemapping\10_paper_writting\figure\v6_250321'
os.makedirs(fig_path,exist_ok=True)
# latlon_1degree_SJ_last_data = gp.read_file(gdb,layer='Global_grid_shp_1degree_hu_Count_Area')

global_diss=r'D:\lakemapping\0_auxiliary_data\World_Continents\world_vector\World_continents_dissolve_except_Antarctica.shp'
Arid_Region=r'D:\lakemapping\0_auxiliary_data\AI\Global-AI_ET0_annual_v3\Global-AI_ET0_v3_annual\Global_AI_Reclassify_Clip_simple.shp'

color_list=['#B8DBB3','#72B063','#E29135','#94C6CD','#719AAC','#4A5F7E']#浅绿，深绿，橙色，浅蓝，中蓝，深蓝
red_color_list=['#311d3f','#522546','#88304e']
label_list=['0.005~0.01 ${km^2}$','0.01~0.03 ${km^2}$','0.03~0.1 ${km^2}$','0.1~1 ${km^2}$','>1 ${km^2}$']
sub_fig_name=['a','b','c','d','e','f']

# lake_category_bins=[0.005,0.01,1,100,10000,1000000]
# lake_category_labels=['Tiny','Small','Medium','Large','Super large']

lake_category_bins=[0.005,0.01,1,100,10000,1000000]
lake_category_labels=['1','3','10','100']

In [ ]:
latlon_1degree_SJ_last_data=gp.read_file(gdb,layer='Global_grid_shp_1degree_hu_Count_Area')
latlon_1degree_SJ_last_data

In [ ]:
Lat_data = latlon_1degree_SJ_last_data.drop(columns='geometry').groupby(['Lat_down']).sum().reset_index()
latitude= Lat_data['Lat_down'].values
Lat_data

In [ ]:
Lon_data = latlon_1degree_SJ_last_data.drop(columns='geometry').groupby(['Lon_left']).sum().reset_index()
longitude= Lon_data['Lon_left'].values
Lon_data

In [ ]:
GLAKES_SJ_last_data=gp.read_file(gdb,layer='Global_grid_shp_1degree_GLAKES_Count_Area')
G_Lat_data = GLAKES_SJ_last_data.drop(columns='geometry').groupby(['Lat_down']).sum().reset_index()
G_Lon_data = GLAKES_SJ_last_data.drop(columns='geometry').groupby(['Lon_left']).sum().reset_index()
G_Lat_data

In [ ]:
def add_background_fig(ax,ax_legend,title,data,terrain_map,column,legend_kwds,norm,grid_flag=True):
    map1 = Basemap( llcrnrlat=-60, urcrnrlat=90, llcrnrlon=-180, urcrnrlon=180,ax=ax)
    map1.readshapefile(global_diss[:-4],'global_diss',drawbounds=True,color='k',linewidth=0.4)
    ax_legend.set_title(title)
    if grid_flag:
        map1.drawmeridians(np.arange(0, 360, 60),labels=grid_labels[k],color='darkgrey', linewidth=0.5,dashes=[5, 5])#绘制经线# left, right, top or bottom   labelstyle='+/-'
        map1.drawparallels(np.arange(-30, 90, 30),labels=[1,1,0,0],color='darkgrey', linewidth=0.5,dashes=[5, 5])#绘制纬线 'darkgrey'
    
    tu=data.plot(
            ax=ax,
            cax=ax_legend,
            zorder=1,
            legend=True,  # 图例
            column=column,  # 分级设色字段#picover0\pia_0
            cmap=terrain_map,#terrain_map,  # 渐变色带的名称#Set2  #seismic
            legend_kwds=legend_kwds,
            norm=norm
            )
def draw_line(ax,xlim,xticks,ylim,yticks,color_list,line_color,line_axes,x,y):
    ax.set_xlim(xlim)
    ax.set_xticks(xticks)
    ax.set_ylim(ylim)
    ax.set_yticks(yticks)
    ax.spines['left'].set_color(color_list[0])
    ax.spines['right'].set_color(color_list[1])
    ax.spines['top'].set_color(color_list[2])
    ax.spines['bottom'].set_color(color_list[3])
    ax.plot(x, y, color=line_color, linestyle = "-",linewidth = 0.8)
    if line_axes !=None:
        ax.tick_params(axis=line_axes, colors=line_color)  # x 轴刻度线和标签颜色
    
def custom_ticks(x, pos):
        if x == 0:
            return "0"
        poww=int(math.log10(x))
        return f"10$^{poww}$"

In [ ]:
latlon_1degree_SJ_last_data['hu_c0'].max()

In [ ]:
# savepic=os.path.join(fig_path,'3-1 global lake distribution.png')
savepic=os.path.join(fig_path,'Figure6 global lake distribution.png')
fig=plt.figure(figsize=(7.23,6.25),dpi=300)#fig=plt.figure(figsize=(18,7.9))
gs = gridspec.GridSpec(4,2,width_ratios=[6,0.8],height_ratios=[2.5,2.5,0.45,0.8],wspace =0.18,hspace =0)

columns=['hu_ap','hu_c0']
grid_labels=[[0,0,1,0],[0,0,0,1]]
titles=['Area density (%)','Count']
colormap=['YlOrRd','YlGnBu']
norms=[colors.PowerNorm(gamma=0.4,vmin=0.0, vmax=20),colors.LogNorm(vmin=3, vmax=30000)]
ticks=[[0,1,10,20],[0,10,100,1000,10000]]

# ticks=[[0,0.5,3,10,50,100],[0,10,100,1000,10000,100000]]
# norms=[]
# for i in [0,1]:
#     cmap=plt.get_cmap(colormap[i])
#     norms.append(colors.BoundaryNorm(ticks[i], cmap.N))
# 
ax2_list=[]
for k in [0,1]:
    ax=plt.subplot(gs[k, 0])
    RdYlBu_r = plt.cm.get_cmap(colormap[k], 256)#
    newcolors = RdYlBu_r(np.linspace(0,1, 256))
    white = np.array([255/256, 255/256, 255/256,1])
    newcolors[:1, :] = white
    terrain_map = ListedColormap(newcolors)
#     terrain_map=plt.get_cmap(colormap[k])
    axins = ax.inset_axes((0.625, 0.095,0.2,0.05)) 
    legend_kwds={'orientation': "horizontal",'extend':'max','ticks': ticks[k]}
#     if k==0:
#         legend_kwds={'orientation': "horizontal",'ticks': ticks[k]}
#     else:
#         legend_kwds={'orientation': "horizontal",'extend':'max','ticks': ticks[k]}
    add_background_fig(ax,axins,titles[k],latlon_1degree_SJ_last_data,terrain_map,columns[k],legend_kwds,norms[k],False)
    ax.text(-200,75,sub_fig_name[k],fontsize=12,weight='bold')
    
    ax.set_xticks(np.arange(-180, 181, 60))
    ax.set_yticks(np.arange(-30, 61, 30))
    ax.tick_params(axis='both', which='major', length=2.5)
    ax.set_xticklabels([f'{abs(x)}°W' if x < 0 else f'{abs(x)}°E' for x in np.arange(-180, 181, 60)],fontsize=6) 
    ax.set_yticklabels([f'{abs(y)}°S' if y < 0 else f'{abs(y)}°N' for y in np.arange(-30, 61, 30)],fontsize=6)
    if k==0:
        ax.tick_params(axis='y',labelright=True)
        ax.tick_params(axis='x',labeltop=True,labelbottom=False)
    else:
        ax.tick_params(axis='y',labelright=True)
    plt.tick_params(bottom=True, top=True, left=True, right=True)

    if k==1:
        axins.xaxis.set_major_formatter(ticker.FuncFormatter(custom_ticks))
    else:
        def custom_ticks_2(x, pos):
            if x==0.5:
                return 0.5
            return int(x)
        axins.xaxis.set_major_formatter(ticker.FuncFormatter(custom_ticks_2))
    ax2_list.append(ax.inset_axes((0.075, 0.1,0.175,0.35)))

data=[[606729.6733,887645.6128,1912565.588],[11758786,201172,1911]]
p=[[17.81,26.05,56.14],[98.3,1.68,0.01]]
yrange=[[100000,2000000],[1000,20000000]]
ytick=[[100000,1000000,2000000],[1000,100000,10000000,20000000]]
ylabel=['Area (km$^2$)','Count']
for i in range(2):
    ax2 = ax2_list[i]
    ax2.tick_params(axis='both', which='major', length=2.1)
    ax2.tick_params(axis='y', which='minor', length=1.5)
    ax2.bar(range(0,3),data[i],0.5,facecolor='w',edgecolor='k',linewidth=0.8)
    ax2.set_xticks(range(0,3))
    ax2.set_xticklabels(['Small','Medium','Large'],fontsize=6)
    ax2.set_yscale('log')
    ax2.set_ylim(yrange[i])
    ax2.set_yticks(ytick[i][:-1])
#     ax2.set_yticklabels(ytick[i],fontsize=6)
    ax2.set_xlim([-0.5,2.5])
    for x in range(3):
        ax2.text(x+0.1,data[i][x]+10,f'{p[i][x]}%',fontsize=6,ha='center',va='bottom')
    ax2.set_ylabel(ylabel[i],fontsize=6, labelpad=0)
    ax2.spines['right'].set_visible(False)
    ax2.spines['top'].set_visible(False)
## 面积、数量纬度分布图
### 面积
ax2=plt.subplot(gs[1, 1],)
plt.tick_params(bottom=True, top=True, left=True, right=True )
plt.tick_params(labelbottom=False,labeltop=True,labelleft=False, labelright=False)
s=4
x=Lat_data['hu_a0'].values/(10.0**s)
# plt.yticks(visible=False)
draw_line(ax2,[0,15],range(0,16,5),[-60,90],range(-60,91,30),['k','k','#4A5F7E','#9e003a'],'#9e003a','x',x,latitude)
ax2.text(14.7,-57,f'×10$^{s}$',ha='right',va='bottom', color='#9e003a')
ax2.set_ylabel('Latitude')
ax2.text(-5,80,sub_fig_name[2],fontsize=12,weight='bold')
ax2.set_xlabel('Area ($\mathregular{km^2}$)', color='#9e003a')
ax22 = ax2.twiny()
s=5
x=Lat_data['hu_c0'].values/(10.0**s)
draw_line(ax22,[0,6],range(0,7,2),[-60,90],range(-60,91,30),['k','k','#4A5F7E','#9e003a'],'#4A5F7E','x',x,latitude)
ax22.text(5.88,87,f'×10$^{s}$',ha='right',va='top', color='#4A5F7E')
ax22.set_xlabel('Count', color='#4A5F7E')


ax3=plt.subplot(gs[3,0])
plt.tick_params(bottom=True, top=True, left=True, right=True )
plt.tick_params(labelbottom=False,labeltop=False,labelleft=True, labelright=False)
ax3.text(-200,2,sub_fig_name[3],fontsize=12,weight='bold')
s=5
y=Lon_data['hu_c0'].values/(10.0**s)
draw_line(ax3,[-180,180],range(-180,181,60),[0,2],[0,1,2],['#4A5F7E','#9e003a','k','k'],'#4A5F7E','y',longitude,y)
ax3.text(-179,1.92,f'×10$^{s}$',ha='left',va='top', color='#4A5F7E')
ax3.set_xlabel('Longitude')
ax3.xaxis.set_label_position("top")
ax3.set_ylabel('Count', color='#4A5F7E')

ax33 = ax3.twinx()
s=4
y=Lon_data['hu_a0'].values/(10.0**s)
draw_line(ax33,[-180,180],range(-180,181,60),[0,10],range(0,11,5),['#4A5F7E','#9e003a','k','k'],'#9e003a','y',longitude,y)
ax33.set_ylabel('Area ($\mathregular{km^2}$)', color='#9e003a')
ax33.text(179,9.6,f'×10$^{s}$',ha='right',va='top', color='#9e003a')
# plt.tight_layout()
plt.savefig(savepic,dpi=400,bbox_inches='tight')#,dpi=400
plt.show()

In [ ]:
def draw_line_2(ax,xlim,xticks,ylim,yticks,line_color,linewidth,label,x,y):
    ax.set_xlim(xlim)
    ax.set_xticks(xticks)
    ax.set_ylim(ylim)
    ax.set_yticks(yticks)
    ax.plot(x, y, color=line_color, linestyle = "-",linewidth = linewidth,label=label)
    
savepic=os.path.join(fig_path,'3-1_global_lake_distribution_small_lake.png')
fig=plt.figure(figsize=(6.18,6.25),dpi=300)#fig=plt.figure(figsize=(18,7.9))
gs = gridspec.GridSpec(4,2,width_ratios=[6,0.8],height_ratios=[0.8,2.5,2.5,0.8],wspace =0.18,hspace =0.18)

columns=['hu_ap','hu_c0']
grid_labels=[[0,0,1,0],[0,0,0,0]]
titles=['Area density (%)','Count']
colormap=['YlOrRd','YlGnBu']
ticks=[[0,0.5,3,10,50,100],[0,10,100,1000,10000,100000]]
norms=[]
for i in [0,1]:
    cmap=plt.get_cmap(colormap[i])
    norms.append(colors.BoundaryNorm(ticks[i], cmap.N))
    
## 面积、数量纬度分布图
### a 经度面积
ax1=plt.subplot(gs[0, 0],)
ax1.text(-175,7.5,sub_fig_name[0],fontsize=12,weight='bold')
s=4
y=Lon_data['hu_a0'].values/(10.0**s)
draw_line_2(ax1,[-180,180],range(-180,181,60),[0,10],[0,5,10],red_color_list[0],0.8,'total',longitude,y)
y=Lon_data['hu_a1'].values/(10.0**s)#GL_c0
draw_line_2(ax1,[-180,180],range(-180,181,60),[0,10],[0,5,10],red_color_list[2],0.6,'area < 0.1 km$^2$',longitude,y)
y=y+Lon_data['hu_a10'].values/(10.0**s)#GL_c0
draw_line_2(ax1,[-180,180],range(-180,181,60),[0,10],[0,5,10],red_color_list[1],0.6,'0.1 ≤ area < 1 km$^2$',longitude,y)
ax1.legend(loc=(1, 0.02),frameon=False)
ax1.text(-180,10,f'×10$^{s}$',ha='left',va='bottom', color='k')
ax1.set_xlabel('Longitude')
ax1.set_ylabel('Area (km$^2$)', color='k')

## b 格网面积
ax=plt.subplot(gs[1, 0])
ax.text(-175,75,sub_fig_name[1],fontsize=12,weight='bold')
terrain_map=plt.get_cmap(colormap[0])
axins = ax.inset_axes((0.03, 0.1,0.22,0.05)) 
legend_kwds={'orientation': "horizontal",'ticks': ticks[0]}
add_background_fig(ax,axins,titles[0],latlon_1degree_SJ_last_data,terrain_map,columns[0],legend_kwds,norms[0],False)
def custom_ticks_2(x, pos):
    if x==0.5:
        return 0.5
    return int(x)
axins.xaxis.set_major_formatter(ticker.FuncFormatter(custom_ticks_2))

## c 纬度面积
ax2=plt.subplot(gs[1, 1])
ax2.text(2,77,sub_fig_name[2],fontsize=12,weight='bold')
s=4
x=Lat_data['hu_a0'].values/(10.0**s)
draw_line_2(ax2,[0,15],[0,5,10,15],[-60,90],range(-60,91,30),red_color_list[0],0.8,'total',x,latitude)
x=Lat_data['hu_a1'].values/(10.0**s)#GL_c0
draw_line_2(ax2,[0,15],[0,5,10,15],[-60,90],range(-60,91,30),red_color_list[2],0.6,'area < 0.1 km$^2$',x,latitude)
x=x+Lat_data['hu_a10'].values/(10.0**s)#GL_c0
draw_line_2(ax2,[0,15],[0,5,10,15],[-60,90],range(-60,91,30),red_color_list[1],0.6,'0.1< area < 0.1 km$^2$',x,latitude)

ax2.set_ylabel('Latitude')
ax2.set_xlabel('Count')
ax2.text(15.5,-60,f'×10$^{s}$',ha='left',va='bottom', color='k')

## d 格网数量
ax3=plt.subplot(gs[2, 0])
ax3.text(-175,75,sub_fig_name[3],fontsize=12,weight='bold')
terrain_map=plt.get_cmap(colormap[1])
axins_3 = ax3.inset_axes((0.03, 0.1,0.22,0.05)) 
legend_kwds={'orientation': "horizontal",'ticks': ticks[1]}
add_background_fig(ax3,axins_3,titles[1],latlon_1degree_SJ_last_data,terrain_map,columns[1],legend_kwds,norms[1],False)
axins_3.xaxis.set_major_formatter(ticker.FuncFormatter(custom_ticks))

### e.纬度数量
ax4=plt.subplot(gs[2, 1],)
ax4.text(0.75,77,sub_fig_name[4],fontsize=12,weight='bold')
s=5
x=Lat_data['hu_c0'].values/(10.0**s)
draw_line_2(ax4,[0,6],range(0,7,2),[-60,90],range(-60,91,30),color_list[-1],0.8,'total',x,latitude)
x=Lat_data['hu_c1'].values/(10.0**s)
draw_line_2(ax4,[0,6],range(0,7,2),[-60,90],range(-60,91,30),color_list[-3],0.6,'area < 0.1 km$^2$',x,latitude)
x=x+Lat_data['hu_c10'].values/(10.0**s)
draw_line_2(ax4,[0,6],range(0,7,2),[-60,90],range(-60,91,30),color_list[-2],0.6,'0.1< area < 0.1 km$^2$',x,latitude)

ax4.set_ylabel('Latitude')
ax4.set_xlabel('Count')
ax4.text(6.2,-60,f'×10$^{s}$',ha='left',va='bottom', color='k')

### f.经度数量
ax5=plt.subplot(gs[3,0])
ax5.text(-175,1.4,sub_fig_name[5],fontsize=12,weight='bold')
s=5
y=Lon_data['hu_c0'].values/(10.0**s)
draw_line_2(ax5,[-180,180],range(-180,181,60),[0,2],[0,1,2],color_list[-1],0.8,'total',longitude,y)
y=Lon_data['hu_c1'].values/(10.0**s)#GL_c0
draw_line_2(ax5,[-180,180],range(-180,181,60),[0,2],[0,1,2],color_list[-3],0.6,'area < 0.1 km$^2$',longitude,y)
y=y+Lon_data['hu_c10'].values/(10.0**s)#GL_c0
draw_line_2(ax5,[-180,180],range(-180,181,60),[0,2],[0,1,2],color_list[-2],0.6,'0.1 ≤ area < 1 km$^2$',longitude,y)
ax5.legend(loc=(1, 0.02),frameon=False)
ax5.text(-180,2,f'×10$^{s}$',ha='left',va='bottom', color='k')
ax5.set_xlabel('Longitude')
ax5.set_ylabel('Count', color='k')
# # plt.tight_layout()
plt.savefig(savepic,dpi=400,bbox_inches='tight')#,dpi=400
plt.show()

In [ ]:
Lat_data.to_excel(r'D:\lakemapping\10_paper_writting\data_lat_distribution.xlsx',index=False)

In [ ]:
Lon_data.to_excel(r'D:\lakemapping\10_paper_writting\data_lon_distribution.xlsx',index=False)